## Import libraries

In [348]:
import warnings
warnings.filterwarnings("ignore")

In [349]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance

## Anomaly-based IDS - Infiltration

### Generate the port-scan datasets for unknown attack detection

In [519]:
df=pd.read_csv('CICIDS2017_sample_km.csv')

In [520]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,2.565000,-0.439152,-0.005817,-0.009418,-0.040156,-0.007139,-0.211382,-0.210703,-0.153699,-0.164849,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
1,2.759600,-0.439341,-0.009819,-0.009418,-0.051263,-0.007139,-0.246240,-0.210703,-0.213347,-0.182201,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
2,2.725909,-0.439340,-0.009819,-0.009418,-0.051863,-0.007139,-0.246240,-0.310140,-0.229468,-0.167112,...,0.002556,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
3,2.967710,-0.439345,-0.009819,-0.010421,-0.051263,-0.007142,-0.246240,-0.210703,-0.213347,-0.182201,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
4,2.683412,-0.430518,-0.008485,-0.008416,-0.045960,-0.007137,-0.180706,-0.210703,-0.151549,-0.097234,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25485,-0.417175,2.529503,0.022195,0.015647,0.612863,-0.007073,1.333548,-0.210703,1.066630,1.056629,...,0.002545,0.847478,0.135514,0.572316,0.898203,1.023440,4.126922,1.900294,0.555568,9
25486,-0.417175,2.871035,0.015525,0.010634,0.403729,-0.007086,0.693545,-0.210703,0.860244,0.661178,...,0.002545,0.773475,-0.099716,0.420533,0.907276,1.967152,-0.109145,1.892086,2.006554,9
25487,-0.417175,1.922710,0.042204,0.028681,1.412575,-0.007038,1.746272,-0.210703,1.609455,1.434458,...,0.002545,4.152361,7.829669,4.707039,0.882819,1.201179,5.539049,1.904398,0.444283,9
25488,-0.417175,1.000343,-0.003150,-0.002400,0.141060,-0.007121,1.053285,-0.210703,1.191111,1.068873,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,9


In [521]:
df.Label.value_counts()

,count
Label,
0,10226
4,4619
10,3178
2,2564
1,1966
12,1507
14,652
3,208
7,155


In [522]:
df1 = df[df['Label'] != 12]
df1['Label'][df1['Label'] > 0] = 1
df1.to_csv('./CICIDS2017_sample_km_without_infiltration.csv',index=0)

In [523]:
df2 = df[df['Label'] == 12]
df2['Label'][df2['Label'] == 12] = 1
df2.to_csv('./CICIDS2017_sample_km_infiltration.csv',index=0)

### Read the generated datasets for unknown attack detection

In [524]:
df1 = pd.read_csv('./CICIDS2017_sample_km_without_infiltration.csv')
df2 = pd.read_csv('./CICIDS2017_sample_km_infiltration.csv')

In [525]:
features = df1.drop(['Label'],axis=1).dtypes[df1.dtypes != 'object'].index
df1[features] = df1[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
df2[features] = df2[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
df1.replace([np.inf, -np.inf], np.nan, inplace=True)
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
df1 = df1.fillna(0)
df2 = df2.fillna(0)

In [526]:
df1.Label.value_counts()

,count
Label,
1,13757
0,10226


In [527]:
df2.Label.value_counts()

,count
Label,
1,1507


In [528]:
df2p=df1[df1['Label']==0]
df2pp=df2p.sample(n=None, frac=1507/10226, replace=False, weights=None, random_state=None, axis=0)
df2=pd.concat([df2, df2pp])

In [529]:
df2.Label.value_counts()

,count
Label,
1,1507
0,1507


In [530]:
df = pd.concat([df1, df2])

In [531]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,3.062168,-0.513386,-0.009198,-0.011629,-0.030942,-0.009852,-0.155204,-0.092884,-0.088204,-0.154906,...,-0.942557,-0.126612,-0.082462,-0.138004,-0.111383,-0.448918,-0.134514,-0.456537,-0.432905,0
1,3.287556,-0.513559,-0.012987,-0.011629,-0.034316,-0.009852,-0.184947,-0.092884,-0.137866,-0.173619,...,-0.942557,-0.126612,-0.082462,-0.138004,-0.111383,-0.448918,-0.134514,-0.456537,-0.432905,0
2,3.248535,-0.513558,-0.012987,-0.011629,-0.034498,-0.009852,-0.184947,-0.204923,-0.151289,-0.157346,...,0.822383,-0.126612,-0.082462,-0.138004,-0.111383,-0.448918,-0.134514,-0.456537,-0.432905,0
3,3.528591,-0.513563,-0.012987,-0.012605,-0.034316,-0.009855,-0.184947,-0.092884,-0.137866,-0.173619,...,-0.942557,-0.126612,-0.082462,-0.138004,-0.111383,-0.448918,-0.134514,-0.456537,-0.432905,0
4,3.199314,-0.505460,-0.011724,-0.010653,-0.032705,-0.009850,-0.129030,-0.092884,-0.086414,-0.081981,...,-0.942557,-0.126612,-0.082462,-0.138004,-0.111383,-0.448918,-0.134514,-0.456537,-0.432905,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7465,3.221865,-0.513562,-0.014250,-0.011629,-0.035440,-0.009855,-0.221828,-0.204923,-0.220637,-0.241424,...,0.822383,-0.126612,-0.082462,-0.138004,-0.111383,-0.448918,-0.134514,-0.456537,-0.432905,0
4728,-0.416593,-0.512924,-0.012987,-0.010653,-0.033434,-0.009756,-0.182567,0.411295,-0.072992,-0.241424,...,0.822383,-0.126612,-0.082462,-0.138004,-0.111383,-0.448918,-0.134514,-0.456537,-0.432905,0
4729,-0.416593,-0.512867,-0.012987,-0.010653,-0.033434,-0.009756,-0.182567,0.411295,-0.072992,-0.241424,...,1.999010,-0.126612,-0.082462,-0.138004,-0.111383,-0.448918,-0.134514,-0.456537,-0.432905,0
6959,-0.391888,-0.443620,-0.005410,-0.001868,0.016349,-0.006786,1.011917,-0.204923,0.732341,1.205194,...,-0.942557,-0.126612,-0.082462,-0.138004,-0.111383,-0.448918,-0.134514,-0.456537,-0.432905,0


In [532]:
X = df.drop(['Label'],axis=1) .values
y = df.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)
pd.Series(y).value_counts()

,count
1,15264
0,11733


### Feature engineering (IG, FCBF, and KPCA)

#### Feature selection by information gain (IG)

In [533]:
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X, y)

In [534]:
# calculate the sum of importance scores
f_list = sorted(zip(map(lambda x: round(x, 4), importances), features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])

In [535]:
# select the important features from top to bottom until the accumulated importance reaches 90%
f_list2 = sorted(zip(map(lambda x: round(x, 4), importances/Sum), features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2>=0.9:
        break

In [536]:
X_fs = df[fs].values

In [537]:
X_fs.shape

(26997, 48)

In [538]:
X_fs

array([[ 3.0621678 , -0.41858864, -0.53441696, ..., -0.13800442,
        -0.25201953, -0.11138313],
       [ 3.28755625, -0.41873098, -0.54471234, ..., -0.13800442,
        -0.25201953, -0.11138313],
       [ 3.24853453, -0.41870626, -0.54251396, ..., -0.13800442,
        -0.25201953, -0.11138313],
       ...,
       [-0.41659322, -0.41805342, -0.51226431, ..., -0.13800442,
        -0.25201953, -0.11138313],
       [-0.39188791, -0.24200126,  0.18380843, ..., -0.13800442,
        -0.16044161, -0.11138313],
       [-0.41659322, -0.41831931, -0.52168817, ..., -0.13800442,
        -0.25201953, -0.11138313]])

#### Feature selection by Fast Correlation Based Filter (FCBF)

The module is imported from the GitHub repo: https://github.com/SantiagoEG/FCBF_module

In [33]:
!pip install scikit-optimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 519.9 kB/s eta 0:00:00


In [539]:
from skopt import gp_minimize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

def fcbf_objective(threshold, X, y, clf=RandomForestClassifier(random_state=42)):
    selector = FCBF(th=threshold)
    X_selected = selector.fit_transform(X, y)
    if X_selected.shape[1] == 0:
        return 1.0
    score = cross_val_score(clf, X_selected, y, cv=3, scoring="accuracy").mean()
    return -score

In [540]:
from FCBF_module import FCBF, FCBFK, FCBFiP, get_i
fcbf = FCBF(th = 0.09988304703442026)

In [541]:
X_fss = fcbf.fit_transform(X_fs,y)

In [542]:
X_fss.shape

(26997, 1)

In [543]:
X_fss

array([[ 3.0621678 ],
       [ 3.28755625],
       [ 3.24853453],
       ...,
       [-0.41659322],
       [-0.39188791],
       [-0.41659322]])

####  kernel principal component analysis (KPCA)

In [ ]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def kpca_objective(params, X, y):
    n_components, kernel = params
    n_components = min(n_components, X.shape[1] - 1)
    if n_components < 1: n_components = 1
    try:
        kpca = KernelPCA(n_components=n_components, kernel=kernel, fit_inverse_transform=False, random_state=42)
        X_kpca = kpca.fit_transform(X)
        clf = RandomForestClassifier(random_state=42)
        score = cross_val_score(clf, X_kpca, y, cv=3, scoring="accuracy").mean()
        return -score
    except Exception as e:
        return 1.0

search_space = [
    Integer(10, 50),
    Categorical(['rbf', 'poly'])
]

result = gp_minimize(
    lambda params: kpca_objective(params, X_fss, y),
    search_space,
    n_calls=20,
    random_state=42,
    acq_func='EI'
)

best_n_components, best_kernel = result.x
print(f"Best n_components: {best_n_components}, Best kernel: {best_kernel}")
print(f"Best accuracy: {-result.fun:.4f}")

In [544]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

kpca = KernelPCA(n_components=20, kernel='rbf', random_state=42)
X_kpca = kpca.fit_transform(X_fss)

In [545]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]

In [546]:
pd.Series(y_train).value_counts()

,count
1,13757
0,10226


In [547]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:15264})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [548]:
pd.Series(y_train).value_counts()

,count
0,15264
1,13757


In [549]:
pd.Series(y_test).value_counts()

,count
1,1507
0,1507


### Apply the cluster labeling (CL) k-means method

In [432]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN,MeanShift
from sklearn.cluster import SpectralClustering,AgglomerativeClustering,AffinityPropagation,Birch,MiniBatchKMeans,MeanShift
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import classification_report
from sklearn import metrics

In [433]:
def CL_kmeans(X_train, X_test, y_train, y_test,n,b=100):
    km_cluster = MiniBatchKMeans(n_clusters=n,batch_size=b)
    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_test)

    count=0
    a=np.zeros(n)
    b=np.zeros(n)
    for v in range(0,n):
        for i in range(0,len(y_train)):
            if result[i]==v:
                if y_train[i]==1:
                    a[v]=a[v]+1
                else:
                    b[v]=b[v]+1
    list1=[]
    list2=[]
    for v in range(0,n):
        if a[v]<=b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0,len(y_test)):
        if result2[v] in list1:
            result2[v]=0
        elif result2[v] in list2:
            result2[v]=1
        else:
            print("-1")
    print(classification_report(y_test, result2))
    cm=confusion_matrix(y_test,result2)
    acc=metrics.accuracy_score(y_test,result2)
    print(str(acc))
    print(cm)

In [550]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:15264})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [551]:
CL_kmeans(X_train, X_test, y_train, y_test, 8)

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1507
           1       0.96      1.00      0.98      1507

    accuracy                           0.98      3014
   macro avg       0.98      0.98      0.98      3014
weighted avg       0.98      0.98      0.98      3014

0.9790975447909754
[[1444   63]
 [   0 1507]]


### Hyperparameter optimization of CL-k-means
Tune "k"

In [552]:
#Hyperparameter optimization by BO-GP
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn import metrics

space  = [Integer(2, 50, name='n_clusters')]
@use_named_args(space)
def objective(**params):
    km_cluster = MiniBatchKMeans(batch_size=100, **params)
    n=params['n_clusters']

    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_val)

    count=0
    a=np.zeros(n)
    b=np.zeros(n)
    for v in range(0,n):
        for i in range(0,len(y_train)):
            if result[i]==v:
                if y_train[i]==1:
                    a[v]=a[v]+1
                else:
                    b[v]=b[v]+1
    list1=[]
    list2=[]
    for v in range(0,n):
        if a[v]<=b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0,len(y_val)):
        if result2[v] in list1:
            result2[v]=0
        elif result2[v] in list2:
            result2[v]=1
        else:
            print("-1")
    cm=metrics.accuracy_score(y_val,result2)
    print(str(n)+" "+str(cm))
    return (1-cm)
from skopt import gp_minimize
import time
t1=time.time()
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
t2=time.time()
print(t2-t1)
print("Best score=%.4f" % (1-res_gp.fun))
print("""Best parameters: n_clusters=%d""" % (res_gp.x[0]))

30 0.8853450072962268
43 0.8915989159891599
43 0.8901396706274755
43 0.8961851156973109
32 0.8928496977277465
20 0.8824265165728581
16 0.8684594538253075
5 0.5188659578903482
15 0.8761726078799249
25 0.8882634980195956
37 0.8855534709193246
50 0.8955597248280175
47 0.8878465707734
50 0.8934750885970398
40 0.8913904523660621
50 0.8953512612049197
50 0.8980612883051907
23 0.8840942255576402
50 0.8934750885970398
34 0.8865957890348134
5.754986763000488
Best score=0.8981
Best parameters: n_clusters=50


In [553]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:15264})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [554]:
CL_kmeans(X_train, X_test, y_train, y_test, res_gp.x[0])

              precision    recall  f1-score   support

           0       1.00      0.81      0.90      1507
           1       0.84      1.00      0.92      1507

    accuracy                           0.91      3014
   macro avg       0.92      0.91      0.91      3014
weighted avg       0.92      0.91      0.91      3014

0.9074319840743198
[[1228  279]
 [   0 1507]]


### Apply the CL-k-means model with biased classifiers

95% of the code has been shared, and the remaining 5% is retained for future extension.  
Thank you for your interest and more details are in the paper.

In [555]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    recall_score
)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np
from skopt import BayesSearchCV
from skopt.space import Integer
from imblearn.over_sampling import SMOTE

def Anomaly_IDS(X_train, X_test, y_train, y_test, n, b=100):
    km = MiniBatchKMeans(n_clusters=n, batch_size=b, random_state=0)
    train_labels = km.fit_predict(X_train)
    test_labels  = km.predict(X_test)

    counts_pos = np.zeros(n)
    counts_neg = np.zeros(n)
    for idx, c in enumerate(train_labels):
        if y_train[idx] == 1: counts_pos[c] += 1
        else:               counts_neg[c] += 1

    cluster_prob = {}
    normal_clusters = []
    attack_clusters = []
    for c in range(n):
        tot = counts_pos[c] + counts_neg[c]
        if counts_pos[c] > counts_neg[c]:
            attack_clusters.append(c)
            cluster_prob[c] = counts_pos[c] / tot if tot>0 else 0.0
        else:
            normal_clusters.append(c)
            cluster_prob[c] = counts_neg[c] / tot if tot>0 else 0.0

    y_km = np.array([1 if c in attack_clusters else 0 for c in test_labels])

    print("CL-k-means Performance:")
    print(classification_report(y_test, y_km))
    cm = confusion_matrix(y_test, y_km)
    tn, fp, fn, tp = cm.ravel()
    dr = tp/(tp+fn) if tp+fn>0 else 0
    far = fp/(fp+tn) if fp+tn>0 else 0
    print(f"  DR: {dr:.4f}, FAR: {far:.4f}\n  CM:\n{cm}\n")

    y_train_km = np.array([1 if c in attack_clusters else 0 for c in train_labels])
    fp_idx = np.where((y_train_km == 1) & (y_train == 0))[0]
    fn_idx = np.where((y_train_km == 0) & (y_train == 1))[0]

    X_fp = X_train[fp_idx]
    X_fn = X_train[fn_idx]
    X_norm = X_train[y_train == 0]
    X_adv  = X_train[y_train == 1]

    if len(X_fp) and len(X_adv):
        attack_samples_for_fp = X_adv[np.random.choice(len(X_adv), size=len(X_fp),replace=True)]
        Xp = np.concatenate([X_fp, attack_samples_for_fp])
        yp = np.concatenate([np.zeros(len(X_fp)), np.ones(len(X_fp))])
        yp = yp.astype(np.int64)
        y_counts = np.bincount(yp)
        print(y_counts)
        min_groups = np.min(y_counts)
        if min_groups >= 2:
          opt_rfp = BayesSearchCV(
              RandomForestClassifier(random_state=0),
              {
                  'n_estimators': Integer(10,200),
                  'max_depth':    Integer(3,50),
                  'min_samples_split': Integer(2,10)
              },
              n_iter=20,
              cv=StratifiedKFold(5, shuffle=True, random_state=0),
              scoring='f1',
              n_jobs=-1,
              random_state=0
          )
          opt_rfp.fit(Xp, yp)
          rfp = opt_rfp.best_estimator_
        else:
          rfp = RandomForestClassifier(random_state=0).fit(X_train, y_train)
    else:
        rfp = RandomForestClassifier(random_state=0).fit(X_train, y_train)

    if len(X_fn) and len(X_norm):
        normal_samples_for_fn = X_norm[np.random.choice(len(X_norm), size=len(X_fn),replace=True)]
        Xn = np.concatenate([X_fn, normal_samples_for_fn])
        yn = np.concatenate([np.zeros(len(X_fn)), np.ones(len(X_fn))])
        yn = yn.astype(np.int64)
        y_counts = np.bincount(yn)
        print(y_counts)
        min_groups = np.min(y_counts)

        if min_groups >= 2:
          opt_rfn = BayesSearchCV(
              RandomForestClassifier(random_state=0),
              {
                  'n_estimators': Integer(10,200),
                  'max_depth':    Integer(3,50),
                  'min_samples_split': Integer(2,10)
              },
              n_iter=20,
              cv=StratifiedKFold(5, shuffle=True, random_state=0),
              scoring='f1',
              n_jobs=-1,
              random_state=0
          )
          opt_rfn.fit(Xn, yn)
          rfn = opt_rfn.best_estimator_
        else:
          rfn = RandomForestClassifier(random_state=0).fit(X_train, y_train)  # fallback
    else:
        rfn = RandomForestClassifier(random_state=0).fit(X_train, y_train)  # fallback

    probs = np.array([cluster_prob.get(c,0.0) for c in test_labels])
    best_thr, best_rec = 0.5, 0.0
    for thr in np.linspace(0.5, 0.99, 50):
        y_tmp = y_km.copy()
        for i, p in enumerate(probs):
            if p < thr:
                if y_tmp[i] == 0:
                    y_tmp[i] = rfn.predict(X_test[i].reshape(1,-1))[0]
                else:
                    y_tmp[i] = rfp.predict(X_test[i].reshape(1,-1))[0]
        rec = recall_score(y_test, y_tmp)
        if rec > best_rec:
            best_rec, best_thr = rec, thr

    y_final = y_km.copy()
    for i, p in enumerate(probs):
        if p < best_thr:
            if y_final[i] == 0:
                y_final[i] = rfn.predict(X_test[i].reshape(1,-1))[0]
            else:
                y_final[i] = rfp.predict(X_test[i].reshape(1,-1))[0]


    print("MTH-IDS Performance:")
    print(classification_report(y_test, y_final))
    cm2 = confusion_matrix(y_test, y_final)

    tn, fp, fn, tp = cm2.ravel()
    dr2 = tp/(tp+fn) if tp+fn>0 else 0
    far2 = fp/(fp+tn) if fp+tn>0 else 0
    acc2 = accuracy_score(y_test, y_final)

    print(f"  Acc: {acc2:.4f}, DR: {dr2:.4f}, FAR: {far2:.4f}\n  CM:\n{cm2}")

    return acc2, dr2, far2, cm2

In [556]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:15264})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [557]:
one_acc, one_dr, one_far, one_f1 = Anomaly_IDS(X_train, X_test, y_train, y_test, res_gp.x[0])

CL-k-means Performance:
              precision    recall  f1-score   support

           0       1.00      0.84      0.91      1507
           1       0.86      1.00      0.93      1507

    accuracy                           0.92      3014
   macro avg       0.93      0.92      0.92      3014
weighted avg       0.93      0.92      0.92      3014

  DR: 1.0000, FAR: 0.1599
  CM:
[[1266  241]
 [   0 1507]]

[2454 2454]
[898 898]
MTH-IDS Performance:
              precision    recall  f1-score   support

           0       1.00      0.84      0.91      1507
           1       0.86      1.00      0.93      1507

    accuracy                           0.92      3014
   macro avg       0.93      0.92      0.92      3014
weighted avg       0.93      0.92      0.92      3014

  Acc: 0.9200, DR: 1.0000, FAR: 0.1599
  CM:
[[1266  241]
 [   0 1507]]
